# DSCI 525: Milestone 1

Group 7:
- Alex Truong Hai Yen
- Peter Yang
- Joshua JiaJie Lim
- Rahul Kuriyedath

In [2]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

In [4]:
#%load_ext rpy2.ipython
%load_ext memory_profiler

## A) Downloading the data

In [6]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareairline/"

In [7]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

In [8]:
%%time
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 6.57 s, sys: 7.44 s, total: 14 s
Wall time: 4min 10s


In [9]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 16.2 s, sys: 2.57 s, total: 18.8 s
Wall time: 21.4 s


## B) Combining data CSVs

In [14]:
import pandas as pd
files = glob.glob('figshareairline/*.csv')
if 'figshareairline/combined_data.csv' or 'figshareairline/observed_daily_rainfall_SYD.csv' in (files):
    files.remove('figshareairline/combined_data.csv')
    files.remove('figshareairline/observed_daily_rainfall_SYD.csv')


In [16]:
%%time
%memit

import pandas as pd # we do not use use_cols trick because we are loading all columns in the file
df = pd.concat((pd.read_csv(file)
                .assign(model=re.findall(r'[^\\/]+(?=_daily_rainfall_NSW)', file)[0])
                for file in files)
              )
df.to_csv("figshareairline/combined_data.csv", index = False)

peak memory: 78.52 MiB, increment: 1.42 MiB
CPU times: user 6min 24s, sys: 32.9 s, total: 6min 57s
Wall time: 7min 14s


In [17]:
%%sh
du -sh figshareairline/combined_data.csv

5.6G	figshareairline/combined_data.csv


In [18]:
%%time
df = pd.read_csv("figshareairline/combined_data.csv")

CPU times: user 57.5 s, sys: 22.8 s, total: 1min 20s
Wall time: 1min 29s


In [19]:
print(df.shape)

(62467843, 7)


In [82]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


**Comparison in run times and memory usages on different machines**

Machine | Peter (Windows 16Gb RAM) |Alex (Windows 8Gb RAM) |Joshua (macOS Catalina 8Gb RAM) |Rahul (Windows 16Gb RAM) 
---|---|---|---|---
Download data - wall timne| 34.1s | 12min 38s | 4min 10s
Unzip data - wall time| 17.8s| 25.5s | 21.4 s
Combine data - wall time| 6min 10s| 8 mins 21s | 7min 14s
Combine data - peak memory | 110.30 MiB| 87.02 MiB| 78.52 MiB
Read combined data - wall time | 53.6s | 1min 44s | 1min 29s

**Comment:**

- Peter's machine can run the task well and faster than Alex's given the bigger RAM

## C) Load combined CSV to memory and perform a simple EDA in Python

### 1. How many records in each type of model?
#### 1.1. Classical way - load all to memory (for comparison later)

In [83]:
%%time
%%memit
df = pd.read_csv("figshareairline/combined_data.csv")
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
TaiESM1             3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
MPI-ESM1-2-LR        966420
AWI-ESM-1-1-LR       966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
peak memory: 2119.57 MiB, increment: 1179.27 MiB
CPU times: user 58.5 s, sys: 19.1 s, total: 1min 17s
Wall time: 1min 23s


#### 1.2. Loading in chunk 

In [84]:
%%time
%%memit
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("figshareairline/combined_data.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int64
peak memory: 1918.27 MiB, increment: 7.45 MiB
CPU times: user 57.6 s, sys: 8.13 s, total: 1min 5s
Wall time: 1min 7s


**Comment:** We cut the peak memory in half by loading the data in chunk as compared to loading all data at once. There is also a minor running time savings of approximately 20-30s. (depends on computer)

### 2. What is the average rain fall (mm/day) for the respective models
#### 2.1. Classical way - load all to memory (for comparison later)

In [85]:
%%time
%%memit
df = pd.read_csv("figshareairline/combined_data.csv")
print(df.groupby(by = ['model']).mean()['rain (mm/day)'])

model
ACCESS-CM2          1.787025
ACCESS-ESM1-5       2.217501
AWI-ESM-1-1-LR      2.026071
BCC-CSM2-MR         1.951832
BCC-ESM1            1.811032
CMCC-CM2-HR4        2.279350
CMCC-CM2-SR5        2.383389
CMCC-ESM2           2.266125
CanESM5             1.894328
EC-Earth3-Veg-LR    1.516258
FGOALS-f3-L         1.627373
FGOALS-g3           2.156419
GFDL-CM4            1.414485
INM-CM4-8           2.811463
INM-CM5-0           2.669012
KIOST-ESM           1.102353
MIROC6              2.301662
MPI-ESM-1-2-HAM     1.610720
MPI-ESM1-2-HR       0.995569
MPI-ESM1-2-LR       1.074308
MRI-ESM2-0          1.368030
NESM3               1.621936
NorESM2-LM          2.230799
NorESM2-MM          2.232966
SAM0-UNICON         2.169676
TaiESM1             2.224576
Name: rain (mm/day), dtype: float64
peak memory: 3459.66 MiB, increment: 2325.96 MiB
CPU times: user 1min 1s, sys: 33.6 s, total: 1min 35s
Wall time: 1min 48s


#### 2.2 Only load the columns that we are interested in

In [86]:
%%time
%%memit
use_cols = ['model','rain (mm/day)']
df = pd.read_csv("figshareairline/combined_data.csv", usecols = use_cols)
print(df.groupby(by = ['model']).mean()['rain (mm/day)'])

model
ACCESS-CM2          1.787025
ACCESS-ESM1-5       2.217501
AWI-ESM-1-1-LR      2.026071
BCC-CSM2-MR         1.951832
BCC-ESM1            1.811032
CMCC-CM2-HR4        2.279350
CMCC-CM2-SR5        2.383389
CMCC-ESM2           2.266125
CanESM5             1.894328
EC-Earth3-Veg-LR    1.516258
FGOALS-f3-L         1.627373
FGOALS-g3           2.156419
GFDL-CM4            1.414485
INM-CM4-8           2.811463
INM-CM5-0           2.669012
KIOST-ESM           1.102353
MIROC6              2.301662
MPI-ESM-1-2-HAM     1.610720
MPI-ESM1-2-HR       0.995569
MPI-ESM1-2-LR       1.074308
MRI-ESM2-0          1.368030
NESM3               1.621936
NorESM2-LM          2.230799
NorESM2-MM          2.232966
SAM0-UNICON         2.169676
TaiESM1             2.224576
Name: rain (mm/day), dtype: float64
peak memory: 2096.42 MiB, increment: 403.25 MiB
CPU times: user 34.1 s, sys: 7.96 s, total: 42.1 s
Wall time: 45.4 s


#### 2.3 Using dask

In [87]:
%%time
%%memit
import dask.dataframe as dd

ddf = dd.read_csv('figshareairline/combined_data.csv', )
print(df.groupby(by = ['model']).mean()['rain (mm/day)'])

model
ACCESS-CM2          1.787025
ACCESS-ESM1-5       2.217501
AWI-ESM-1-1-LR      2.026071
BCC-CSM2-MR         1.951832
BCC-ESM1            1.811032
CMCC-CM2-HR4        2.279350
CMCC-CM2-SR5        2.383389
CMCC-ESM2           2.266125
CanESM5             1.894328
EC-Earth3-Veg-LR    1.516258
FGOALS-f3-L         1.627373
FGOALS-g3           2.156419
GFDL-CM4            1.414485
INM-CM4-8           2.811463
INM-CM5-0           2.669012
KIOST-ESM           1.102353
MIROC6              2.301662
MPI-ESM-1-2-HAM     1.610720
MPI-ESM1-2-HR       0.995569
MPI-ESM1-2-LR       1.074308
MRI-ESM2-0          1.368030
NESM3               1.621936
NorESM2-LM          2.230799
NorESM2-MM          2.232966
SAM0-UNICON         2.169676
TaiESM1             2.224576
Name: rain (mm/day), dtype: float64
peak memory: 1933.41 MiB, increment: 922.60 MiB
CPU times: user 3.94 s, sys: 1.16 s, total: 5.11 s
Wall time: 7.5 s


**Comment:** (All results are computer dependent)
- By loading only the columns of interest instead of all the columns in the dataframe, we can save peak memory usage from 4531.16 MiB in 2.1 to 3982.58 MiB in 2.2. Running time is significantly cut down from 2min 17s to 52.1 s.

- With dask where chunking and parallel execution are applied automatically for us, we can process the data much faster at 5.83s and it also takes up the least amount of memory during peak usage at 2571.79 MiB, that is only 56.8% of what the traditional Pandas method consumes.

## D) Transfer datafram from python to R and Perform a simple EDA in R